In [1]:
# setting up all of the packages!
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import torch
import torch.nn as nn

import rasterio
import rioxarray as rio
import numpy as np
import pandas as pd

#from utils import * 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

## Import and prepare our predictor and predictant datasets
predictor: (X, TerraClim climate variables), predictant: (Y, MODIS LAI) data

In [2]:
# establish path and open as an array
path = "/home/jovyan/large_files/climLai_merged.nc"
ds_combined = xr.open_dataset(path)

In [3]:
# take a look at the ds_combined dataset
print(ds_combined)

<xarray.Dataset> Size: 181MB
Dimensions:      (time: 228, lat: 107, lon: 116)
Coordinates:
  * time         (time) datetime64[ns] 2kB 2002-01-31 2002-02-28 ... 2020-12-31
  * lat          (lat) float32 428B -9.125 -9.175 -9.225 ... -14.38 -14.43
  * lon          (lon) float32 464B -73.47 -73.42 -73.38 ... -67.78 -67.72
Data variables:
    spatial_ref  int64 8B ...
    tmmx         (time, lat, lon) float64 23MB ...
    tmmn         (time, lat, lon) float64 23MB ...
    pr           (time, lat, lon) float64 23MB ...
    pdsi         (time, lat, lon) float64 23MB ...
    def          (time, lat, lon) float64 23MB ...
    vpd          (time, lat, lon) float64 23MB ...
    soil         (time, lat, lon) float64 23MB ...
    lai          (time, lat, lon) float64 23MB ...


In [4]:
# the data array has 3 dimensions. Stack the coordinate values to go from 3d to 2d.
ds_stacked = ds_combined.stack(stacked=("lat", "lon"))

In [5]:
# convert to data frame
df_combined = ds_stacked.to_dataframe()

In [6]:
# from dataframe generate the x_df (ONLY climate predictor variables, drop the lai column)
x_df = df_combined.drop(columns=["lai","spatial_ref"])

In [7]:
# look at x_df
x_df.head()

tmmx  tmmn     pr  pdsi  def   vpd   soil
time       lat       lon                                                  
2002-01-31 -9.125002 -73.474998  32.1  22.2  359.0 -1.68  0.0  0.89  156.8
                     -73.424995  32.0  22.2  361.0 -1.65  0.0  0.88  156.0
                     -73.375000  31.9  22.1  370.0 -1.57  0.0  0.87  154.5
                     -73.324997  31.9  22.1  371.0 -1.52  0.0  0.86  153.7
                     -73.275002  31.8  22.1  375.0 -1.48  0.0  0.85  152.9

In [8]:
# from dataframe generate y_df (ONLY lai)
y_df = df_combined.drop(columns=["spatial_ref",
                                 "tmmx",
                                 "tmmn",
                                 "pr",
                                 "pdsi",
                                 "def",
                                 "vpd",
                                 "soil"])
# look at y_df
y_df.head()

lai
time       lat       lon                 
2002-01-31 -9.125002 -73.474998  4.803474
                     -73.424995  5.169796
                     -73.375000  5.366670
                     -73.324997  5.473258
                     -73.275002  5.366331

## Split our data into training and validation
Look at the entire time sequence of our data: 19 years, 12 months per year
(19yrs x 12months) = 228 time steps

The first 15 years are our training data: 2002 - 2016 (15yrs x 12months = 180 ts)
The last 4 years are our validation data: 2017 - 2020 (4yrs x 12 months = 48 ts)

In [9]:
# look at the entire time sequence of our data
first_time = x_df.index.get_level_values("time")[0]
last_time  = x_df.index.get_level_values("time")[-1]

print('time 0 =', first_time)
print('time last =', last_time)

time 0 = 2002-01-31 00:00:00
time last = 2020-12-31 00:00:00


In [10]:
# split the data according to time splits above

# establish the before and after
times = x_df.index.get_level_values("time")
mask_before = times < '2017-01-31 00:00:00'
mask_after  = times >= '2017-01-31 00:00:00'

# x training validation data split
x_df_train = x_df.loc[mask_before]
x_df_valid  = x_df.loc[mask_after]

# y training validation data split
y_df_train = y_df.loc[mask_before]
y_df_valid = y_df.loc[mask_after]

In [11]:
# test to make sure the splits worked:
print('shape x_df_train', x_df_train.shape)
print('shape x_df_valid', x_df_valid.shape)

print('shape y_df_train', y_df_train.shape)
print('shape y_df_valid', y_df_valid.shape)

shape x_df_train (2234160, 7)
shape x_df_valid (595776, 7)
shape y_df_train (2234160, 1)
shape y_df_valid (595776, 1)


# SKIP THIS! WE SCALE LATER ON! AND BECAUSE WE ARE STANDARDIZING ON SEPERATE TRAINING AND VALIDATION WE ARE INTRODUCING BIAS. THIS IS WRONG. 
## Standardize our data to prepare it for LSTM ingestion

Standardize our data to prepare it for LSTM ingestion :o
We want to standardize per location, to avoid losing any spatial relationships

There is a question here of whether we should be standardizing the y values because NDVI is already a constrained abd bounded value <-- much to think about

In [36]:
# standardizing our y_df_train values
def standardize_per_location(y_df_train, feature_col="lai"):
    # Extract levels
    idx = y_df_train.index

    # Create a DataFrame from index levels for grouping
    idx_df = pd.DataFrame({
        'time': idx.get_level_values('time'),
        'lat': idx.get_level_values('lat'),
        'lon': idx.get_level_values('lon'),
        feature_col: y_df_train[feature_col].values
    })

    # Group by location (lat, lon)
    def standardize_group(group):
        vals = group[feature_col]
        standardized_vals = (vals - vals.mean()) / vals.std()
        group[feature_col] = standardized_vals
        return group

    y_df_train_sd = idx_df.groupby(['lat', 'lon']).apply(standardize_group)

    # Restore MultiIndex
    y_df_train_sd.set_index(['time', 'lat', 'lon'], inplace=True)
    y_df_train_sd = y_df_train_sd.sort_index()

    return y_df_train_sd[[feature_col]]

# Apply
y_df_train_sd = standardize_per_location(y_df_train, feature_col="lai")

/tmp/ipykernel_1065/301446771.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_df_train_sd = idx_df.groupby(['lat', 'lon']).apply(standardize_group)


In [37]:
# standardizing our y_df_valid values
def standardize_per_location(y_df_valid, feature_col="lai"):
    # Extract levels
    idx = y_df_valid.index

    # Create a DataFrame from index levels for grouping
    idx_df = pd.DataFrame({
        'time': idx.get_level_values('time'),
        'lat': idx.get_level_values('lat'),
        'lon': idx.get_level_values('lon'),
        feature_col: y_df_valid[feature_col].values
    })

    # Group by location (lat, lon)
    def standardize_group(group):
        vals = group[feature_col]
        standardized_vals = (vals - vals.mean()) / vals.std()
        group[feature_col] = standardized_vals
        return group

    y_df_valid_sd = idx_df.groupby(['lat', 'lon']).apply(standardize_group)

    # Restore MultiIndex
    y_df_valid_sd.set_index(['time', 'lat', 'lon'], inplace=True)
    y_df_valid_sd = y_df_valid_sd.sort_index()

    return y_df_valid_sd[[feature_col]]

# Apply
y_df_valid_sd = standardize_per_location(y_df_valid, feature_col="lai")

/tmp/ipykernel_1065/680808079.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_df_valid_sd = idx_df.groupby(['lat', 'lon']).apply(standardize_group)


In [38]:
def standardize_per_location(df, feature_cols):
    """
    Standardize multiple feature columns per location (lat, lon) over time.

    Parameters:
    - df: pandas DataFrame with MultiIndex (time, lat, lon)
    - feature_cols: list of feature column names to standardize

    Returns:
    - pandas DataFrame with standardized features, same MultiIndex
    """

    idx = df.index

    # Build a DataFrame from the MultiIndex levels and feature columns
    data = {
        'time': idx.get_level_values('time'),
        'lat': idx.get_level_values('lat'),
        'lon': idx.get_level_values('lon')
    }
    # Add all feature columns to the data dict
    for col in feature_cols:
        data[col] = df[col].values

    idx_df = pd.DataFrame(data)

    # Group by location (lat, lon) and standardize each feature column
    def standardize_group(group):
        for col in feature_cols:
            vals = group[col]
            group[col] = (vals - vals.mean()) / vals.std()
        return group

    standardized_df = idx_df.groupby(['lat', 'lon']).apply(standardize_group)

    # Restore MultiIndex and sort
    standardized_df.set_index(['time', 'lat', 'lon'], inplace=True)
    standardized_df = standardized_df.sort_index()

    # Return only the standardized feature columns (MultiIndex preserved)
    return standardized_df[feature_cols]


feature_columns = ["tmmx", "tmmn", "pr", "pdsi", "def", "vpd", "soil"]  # your features
x_df_train_sd = standardize_per_location(x_df_train, feature_columns)
x_df_valid_sd = standardize_per_location(x_df_valid, feature_columns)

/tmp/ipykernel_1065/2995428060.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  standardized_df = idx_df.groupby(['lat', 'lon']).apply(standardize_group)
/tmp/ipykernel_1065/2995428060.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  standardized_df = idx_df.groupby(['lat', 'lon']).apply(standardize_group)


## Reshape data to feed into the LSTM model
okay let's do this!! >:O

In [12]:
# import our missing packages!
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader

In [13]:
# 0 - Ensure Time is a column of our data
# Time is currently an INDEX not a column of our data, we need to convert it into a COLUMN
def ensure_time_column(df):
    """
    If time is the index, convert it into a column called 'time'.
    Works for both training and validation dataframes.
    """
    if "time" not in df.columns:
        # index must be time
        df = df.reset_index().rename(columns={"index": "time"})
    df["time"] = pd.to_datetime(df["time"])
    return df


# 1 - Merge the X (predictors, climate variables) and Y (predictands, modis lai) datasets
def merge_xy(x_df, y_df):
    x_df = ensure_time_column(x_df)
    y_df = ensure_time_column(y_df)
    
    df = x_df.merge(y_df, on=["time", "lat", "lon"], how="inner")
    df = df.sort_values(["lat", "lon", "time"]).reset_index(drop=True)
    return df

train_df = merge_xy(x_df_train, y_df_train)
valid_df = merge_xy(x_df_valid, y_df_valid)

In [14]:
# 2. scale the climate inputs, DO NOT DO THIS IF YOU HAVE PROPERLY STANDARDIZED
# in this case we have not properly standardized, so we will scale
# find the mean and std of each feature in the training dataset and standardize
# then use the mean and std of the TRAINING data to transform the validation data
feature_cols = ["tmmx", "tmmn", "pr", "pdsi", "def", "vpd", "soil"]

scaler = StandardScaler()
train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols]) # TRAINING DATA, CALL THE FIT_TRANSFORM
valid_df[feature_cols] = scaler.transform(valid_df[feature_cols]) # VALIDATION DATA, CALL THE TRANSFORM

In [15]:
# 3 build the lstm sequences for each pixel, establish 6 timesteps (months) as our "look-back" window
def build_sequences(df, feature_cols, lookback):
    X_list, y_list, coords = [], [], []

    for (lat, lon), group in df.groupby(["lat", "lon"]):
        group = group.sort_values("time")

        X_vals = group[feature_cols].values
        y_vals = group["lai"].values

        for i in range(len(group) - lookback):
            X_list.append(X_vals[i:i+lookback])
            y_list.append(y_vals[i+lookback])
            coords.append((lat, lon))

    return (
        np.array(X_list, dtype=np.float32),
        np.array(y_list, dtype=np.float32),
        coords,
    )


lookback = 6 # will use the past 6 months (time-steps of data) to get the next 7th month

X_train_np, y_train_np, train_coords = build_sequences(train_df, feature_cols, lookback)
X_valid_np, y_valid_np, valid_coords = build_sequences(valid_df, feature_cols, lookback)

print("Train X shape:", X_train_np.shape) # (len_t), lookback, feat
print("Valid X shape:", X_valid_np.shape) # (len_v), lookback, feat

# Train X shape and Valid X shape should have the same lookback and features (the last two values)
# BUT should differ in their first domension, with Train beging larger reflecting there is a greater ammount of data going into training

Train X shape: (2159688, 6, 7)
Valid X shape: (521304, 6, 7)


In [16]:
# 4 convert to PYTORCH dataset, ready for model ingestion
class ClimateLAIDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = ClimateLAIDataset(X_train_np, y_train_np)
valid_dataset = ClimateLAIDataset(X_valid_np, y_valid_np)


In [17]:
# 5 split into training, validation, testing data
# current data is split 80% to training, 20% to validation
# we will split the 20% of validation into 10% validation, 10% testing

# Split our validation into valid and test datasets
#valid_size = int(0.1 * len(valid_dataset))
#test_size = len(valid_dataset) - valid_size
#valid_dataset, test_dataset = torch.utils.data.random_split(valid_dataset, [valid_size, test_size])

In [17]:
# 6 DATALOADERS, both should be shuffle=false, according to Pierre
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

## Define LSTM structure

In [18]:
# set hyperparameters
n_neuron       = 64
activation     = 'ReLU'
num_epochs     = 50
learning_rate  = 0.001
minibatch_size = 64
model_num      = 1

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
# model definition
import torch.nn as nn

class LAI_LSTM(nn.Module):
    def __init__(self, num_features=7, hidden_size=64, num_layers=1):
        super(LAI_LSTM, self).__init__()

        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Debug print statements
        #print("\n===== FORWARD PASS DEBUG =====")
        #print("Input to LSTM:", x.shape)

        # Run LSTM
        output, (h_n, c_n) = self.lstm(x)
        #print("LSTM output:", output.shape)      # (batch, seq_len, hidden)
        #print("h_n:", h_n.shape)                 # (num_layers, batch, hidden)
        #print("c_n:", c_n.shape)

        # Fully connected layer on last hidden state
        out = self.fc(h_n[-1])
        #print("After FC:", out.shape)            # (batch, 1)

        # Squeeze last dimension only
        out = out.squeeze(1)
        #print("Final output:", out.shape)        # (batch,)

        #print("===== END FORWARD DEBUG =====\n")
        return out

model = LAI_LSTM(num_features=7, hidden_size=64, num_layers=1)
model.to(device)

LAI_LSTM(
  (lstm): LSTM(7, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [49]:
# define the optimizer (ADAM), and the evaluation criterion: MSELoss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

## debugging, ignore! DO NOT RUN

In [21]:
print("X_train shape:", X_train_np.shape)
print("y_train shape:", y_train_np.shape)

X_train shape: (2159688, 6, 7)
y_train shape: (2159688,)


In [22]:
## de-bugging!! ignore
# ----- RUN ONE BATCH ONLY -----
X_batch, y_batch = next(iter(train_loader))

print("Batch X:", X_batch.shape)
print("Batch y:", y_batch.shape)

_ = model(X_batch)

Batch X: torch.Size([32, 6, 7])
Batch y: torch.Size([32])


In [69]:
# debugging training loop!! ignore!!
for X_batch, y_batch in train_loader:
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    optimizer.zero_grad()

    outputs = model(X_batch)

    print("outputs:", outputs.shape)
    print("y_batch:", y_batch.shape)

    loss = criterion(outputs, y_batch)
    loss.backward()
    optimizer.step()

    print("LOSS INPUT SHAPES →", outputs.shape, y_batch.shape)
    
    break 


===== FORWARD PASS DEBUG =====
Input to LSTM: torch.Size([32, 6, 7])
LSTM output: torch.Size([32, 6, 64])
h_n: torch.Size([1, 32, 64])
c_n: torch.Size([1, 32, 64])
After FC: torch.Size([32, 1])
Final output: torch.Size([32])
===== END FORWARD DEBUG =====

outputs: torch.Size([32])
y_batch: torch.Size([32])
LOSS INPUT SHAPES → torch.Size([32]) torch.Size([32])


## Training and saving the model

In [ ]:
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LAI_LSTM(num_features=7, hidden_size=64, num_layers=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):

    # ===== TRAINING =====
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)        # (batch, seq, features)
        y_batch = y_batch.to(device)        # (batch,)

        optimizer.zero_grad()

        y_pred = model(X_batch)             # → (batch,)
        loss = criterion(y_pred, y_batch)   # matches perfectly
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * len(X_batch)

    train_loss /= len(train_loader.dataset)

    # ===== VALIDATION =====
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in valid_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_loss += loss.item() * len(X_batch)

    val_loss /= len(valid_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}  "
          f"| Train Loss: {train_loss:.4f}  "
          f"| Val Loss: {val_loss:.4f}")

Epoch 1/50  | Train Loss: 0.3611  | Val Loss: 1.8247


In [ ]:
plot_history(train_losses_mse, val_losses_mse)

In [ ]:
# export and SAVE model
model_path = os.path.join(cwd,'saved_model_X')
make_dir(model_path)

# Save the model weights to a pth file.
torch.save(model.state_dict(), os.path.join(model_path,'LSTM_model_weights_.pth'))

#### MISSING TASKS/NEXT STEPS
## 1 figure out data splitting
Currently, I am splitting our data into training and validation at the start of the script and using the timesteps as my splits to have 15 years of training data, 4 years of validation data, which can then be split into validation and training data
The example LSTM script brings in the training and testing data separatley, and then gets the validation data from the training data...
So, I am unsure if I should:
a) go back to the start of the script and split 90% to training, 10% to testing, and then from that 90% generate my training data after all the processing
b) Leave my splitting as is, and split my validation data (currently 20% of my data) into 10% validation, 10% testing
Note that option b) means that the testing data has been standardized/scaled alongside the validation data. I am unclear if this is an issue.

## 2 define the model architecture by setting hyperparameters

## 3 actually define the model and the optimizer (ADAM)

## 4 run the model and save!

## evaluate the trained model on the testing data

